In [1]:
pip install easyocr

  Using cached easyocr-1.7.2-py3-none-any.whl.metadata (10 kB)
  Using cached torch-2.7.0-cp312-cp312-win_amd64.whl.metadata (29 kB)
  Using cached torchvision-0.22.0-cp312-cp312-win_amd64.whl.metadata (6.3 kB)
  Using cached opencv_python_headless-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached python_bidi-0.6.6-cp312-cp312-win_amd64.whl.metadata (5.0 kB)
  Using cached shapely-2.1.0-cp312-cp312-win_amd64.whl.metadata (7.0 kB)
  Using cached pyclipper-1.3.0.post6-cp312-cp312-win_amd64.whl.metadata (9.2 kB)
  Using cached ninja-1.11.1.4-py3-none-win_amd64.whl.metadata (5.0 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
Using cached easyocr-1.7.2-py3-none-any.whl (2.9 MB)
Using cached torchvision-0.22.0-cp312-cp312-win_amd64.whl (1.7 MB)
Using cached torch-2.7.0-cp312-cp312-win_amd64.whl (212.5 MB)
Using cached ninja-1.11.1.4-py3-none-win_amd64.whl (296 kB)
   ---------------------------------------- 0.0/39.4 MB ? eta -:--:--
   ---------------------

ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified



In [ ]:
import cv2
import numpy as np

def detect_number_plate(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply bilateral filter to preserve edges
    filtered = cv2.bilateralFilter(gray, 11, 17, 17)

    # Edge detection
    edged = cv2.Canny(filtered, 30, 200)

    # Find contours
    contours, _ = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:30]

    plate_candidate = None

    for cnt in contours:
        approx = cv2.approxPolyDP(cnt, 0.018 * cv2.arcLength(cnt, True), True)

        if len(approx) == 4:  # Plate is usually a rectangle
            x, y, w, h = cv2.boundingRect(approx)
            aspect_ratio = w / float(h)
            area = cv2.contourArea(cnt)

            # Filter for typical number plate shape and size
            if 2 < aspect_ratio < 6 and area > 1000:
                plate_candidate = approx
                break

    if plate_candidate is not None:
        cv2.drawContours(frame, [plate_candidate], -1, (0, 255, 0), 3)
        x, y, w, h = cv2.boundingRect(plate_candidate)
        cv2.putText(frame, "Number Plate", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)

    return frame

# Open webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame.")
        break

    result_frame = detect_number_plate(frame)
    cv2.imshow("Number Plate Detection", result_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
